# Fetch a full Open-FF data set from pickles

In [ ]:
import pandas as pd
import zipfile
import requests
from io import BytesIO

c = requests.get('https://storage.googleapis.com/gwa-test/sm_dataframe.zip').content

In [ ]:
filebytes = BytesIO(c)
zfile = zipfile.ZipFile(filebytes)
zfile.namelist()

In [ ]:
# now unzip and create the associated dataframes
with zfile.open('repo_info.txt') as f:
    txt = f.read()
print(f' reading data from repository: {txt}')

with zfile.open('disclosures.csv') as f:
    print(f' -- processing disclosures')
    ddf = pd.read_csv(f,quotechar='$',encoding='utf-8',low_memory=False,
                     dtype={'APINumber':'str'})
    ddf.date = pd.to_datetime(ddf.date)
with zfile.open('chemrecs.csv') as f:
    print(f' -- processing chemical records')
    cdf = pd.read_csv(f,quotechar='$',
                      encoding='utf-8',low_memory=False)
mg = pd.merge(cdf,ddf, on='UploadKey', how='left')
with zfile.open('bgCAS.csv') as f:
    print(f' -- processing chemical metadata')
    casdf = pd.read_csv(f,quotechar='$',
                      encoding='utf-8',low_memory=False)
mg = pd.merge(mg,casdf, on='bgCAS', how='left')

mg.info()

